In [ ]:
from pycelonis import get_celonis, Config as celonis_config
from pycelonis.ems import ColumnTransport, ColumnType

In [ ]:
# DO NOT CHANGE ANYTHING
DATA_POOL_ID = '4a953afc-51ae-4198-aa96-15b40d32f004'
DATA_MODEL_ID = 'b3eb41a7-6404-4124-a04a-487b6563f214'
API_TOKEN = 'ZDhkZjY4NzctMjg4OC00MmI2LThlMjUtZTg5Mzk2NGZjNDU5OjVla1JTUkk5ZGZzRlpsTGFxL0NBTnE5bjJINzQ2K0M0RDdzWDBjVDlFWFpq'

celonis = get_celonis(api_token=API_TOKEN, permissions=False)  #key_type='USER_KEY'
data_pool = celonis.data_integration.get_data_pool(DATA_POOL_ID)
data_model = data_pool.get_data_model(DATA_MODEL_ID)

In [5]:
def get_max_field_length(field: pd.Series) -> int:
    max_len = max(len(e) for e in field)
    if not max_len:
        max_len = 1
    next_pow = 2 ** math.ceil(math.log2(max_len))
    return next_pow if next_pow > 80 else 80

column_config = []

df = df.astype(str)

for e in df.columns:
    column_config.append(
        ColumnTransport(
            column_name=e,
            field_length=get_max_field_length(df[e]),
        )
    )

In [ ]:
# push table to data pool
table_name = 'DOC_EXTRACTION_' + MY_NAME

data_pool.create_table(
    df,
    column_config=column_config,
    table_name=table_name,
    drop_if_exists=True,
    force=True,
)

print('Created table ',table_name)

In [13]:
# duplicate data model
try: 
    payload = celonis.client.request(
        url=f'/integration/api/pools/{data_pool.id}/data-models/{data_model.id}/duplicate',
        method='POST',
        parse_json=True
    )
    payload['name'] = data_model.name + '_' + MY_NAME
    print(f"Data model duplicated")
except: 
    print(f"An error occurred while duplicating data model: {e}")

# rename data model
try:
    payload = celonis.client.request(
        url=f'/integration/api/pools/{data_pool.id}/data-models/{payload['id']}',
        method='PUT',
        json = payload,
        parse_json=True
    )
    print(f"Data model renamed to {payload['name']}")
except e: 
    print(f"An error occurred while renaming data model: {e}")

data_model = data_pool.get_data_model(payload['id'])

# add table to DM
try:
    try:
        # Check if the table already exists
        dmt = data_model.get_tables().find_all(table_name)[0]
        dmt.update()
        print(f"Table '{table_name}' updated in Data Model.")
    except IndexError:  # If table not found
        # Add table to Data Model
        data_model.add_table(table_name)
        print(f"Table '{table_name}' added to Data Model.")
except Exception as e:
    print(f"An error occurred while adding or updating tables in the Data Model: {e}")

Data model duplicated
Data model renamed to Challenge_Procurement_NAME_SURNAME


In [16]:
fk_mapping = [("CASE_TABLE", table_name, [("_CASE_KEY", "po_reference")])]

# Get all tables from the data model once and create a name-to-object map for efficient lookup
all_dm_tables_list = data_model.get_tables()
table_map_by_name = {table.name: table for table in all_dm_tables_list}

# Get existing foreign key pairs (source_table_id, target_table_id) once
existing_fk_pairs = set()
try:
    for fk in data_model.get_foreign_keys():
        existing_fk_pairs.add((fk.source_table_id, fk.target_table_id))
except Exception as e:
    print(f"Warning: Could not retrieve existing foreign keys: {e}. Proceeding without this check for all FKs.")

for source_table_name, target_table_name, columns_to_link in fk_mapping:
    # Find source and target table objects using the map
    source_dm_table = table_map_by_name.get(source_table_name)
    target_dm_table = table_map_by_name.get(target_table_name)

    # Validate that tables exist
    if not source_dm_table:
        print(f"Warning: Source table '{source_table_name}' not found in data model. Skipping FK creation to '{target_table_name}'.")
        continue
    if not target_dm_table:
        print(f"Warning: Target table '{target_table_name}' not found in data model. Skipping FK creation from '{source_table_name}'.")
        continue

    # Check if a foreign key already exists between these two tables
    if (source_dm_table.id, target_dm_table.id) not in existing_fk_pairs:
        try:
            print(f"Creating FK from '{source_table_name}' (ID: {source_dm_table.id}) "
                  f"to '{target_table_name}' (ID: {target_dm_table.id}) "
                  f"on columns: {columns_to_link}")
            
            # Create the foreign key
            new_fk = data_model.create_foreign_key(
                source_table_id=source_dm_table.id,
                target_table_id=target_dm_table.id,
                columns=columns_to_link
            )
            print(f"Successfully created FK: {new_fk.id if new_fk else 'N/A'}")
            
            # Add the newly created FK pair to our set to avoid attempting to re-create it
            # if the mapping contains duplicate logical FKs processed in the same run.
            existing_fk_pairs.add((source_dm_table.id, target_dm_table.id))

        except Exception as e:
            # Catching generic Exception, but you might want to catch specific pycelonis exceptions
            print(f"Error creating FK from '{source_table_name}' to '{target_table_name}': {e}")

Creating FK from 'CASE_TABLE' (ID: 85d43687-e2e4-47d7-8489-92789abc6e53) to 'DOC_EXTRACTION_NAME_SURNAME' (ID: 5c06ca2b-c4ac-4c24-80bb-72a72a033681) on columns: [('_CASE_KEY', 'po_reference')]
[2025-07-22 16:02:48,009] INFO: Successfully created foreign key with id '16a9ed11-cfea-4ed4-b4b9-7c780107f1a1'
Successfully created FK: 16a9ed11-cfea-4ed4-b4b9-7c780107f1a1


In [76]:
# find member 
try:
    response = celonis.client.request(
            url=f'/api/members?includeInternal=true',
            method='GET',
            parse_json=True
        )
    member = [ member for member in response if member['email'] == MY_EMAIL][0]
    print(f"Found member with email:", MY_EMAIL)
except Exception as e:
    print(f"Could not find email {MY_EMAIL}, check the variable MY_EMAIL again. Error: ",e)

# give permissions to DM

Found member with email: a.dawson@celonis.com


In [78]:
# give member permissions to DM

try:
    payload = [
        {
            "objectId": data_model.id,
            "subjectId": member['membershipId'],
            "subjectType": "USER",
            "permissions": ["USE"],
            "serviceName": "event-collection"
        }
    ]
    
    response = celonis.client.request(
            url=f'/integration/api/data-models/manage-permissions/{data_model.id}',
            method='POST',
            json=payload,
            parse_json=True
        )
    print(f"Gave permissions to {data_model.name} to member with email:", MY_EMAIL)
except Exception as e:
    print(f"Could not give permissions email {MY_EMAIL}.")

Gave permissions to Challenge_Procurement_NAME_SURNAME to member with email: a.dawson@celonis.com


In [17]:
data_model.reload(wait=True)

[2025-07-22 16:03:14,878] INFO: Successfully triggered data model reload for data model with id '2559ce5f-3926-48d8-9e4a-0a4c2baeda34'
[2025-07-22 16:03:14,878] INFO: Wait for execution of data model reload for data model with id '2559ce5f-3926-48d8-9e4a-0a4c2baeda34'


0it [00:00, ?it/s]

[2025-07-22 16:03:29,132] WARNING: WARNING: The join between table "CASE_TABLE" and table "DOC_EXTRACTION_NAME_SURNAME" did not find any matching rows. Please check your join definition and your data.
